In [1]:
from funciones import Functions
import ipywidgets as widgets
from IPython.display import display
import requests
import pandas as pd
import matplotlib.pyplot as plt

employee_name_widget = widgets.Label(value='Bienvenido al Sistema Analítico Hotelero')
#section_titles = [(id, Titulo),
#                  (id, Titulo)]
section_titles = [("login", "Login Section"),
              ("local_statistics", "Local Statistics"),
              ("global_statistics", "Global Statistics"),
              ("create_records", "Create Records"),
              ("update_delete_records", "Update Delete Records")]

Functions().create_navBar(section_titles)

HTML(value='\n        <style>\n            #navbar {\n              background-color: #FF5733; /* Color de fon…

In [2]:
# Bienvenido al Sistema Analítico Hotelero
#print(f"Bienvenido {employee_name_widget.value}al Sistema Analítico Hotelero ")
display(employee_name_widget)


Label(value='Bienvenido al Sistema Analítico Hotelero')

In [3]:
global_statistic_widget = widgets.Output()
def mostrar_graficos_most_reservation():
    with global_statistic_widget:
        url = 'http://127.0.0.1:5000/most/reservation'
        data = {
            "eid": 3
        }
        response = session.get(url, json=data, verify=False)
        # Verifica si la solicitud fue exitosa
        if response.status_code == 200:
            #print("Respuesta exitosa")
            data_json = response.json()
            # Convierte los datos JSON en un DataFrame de pandas
            df = pd.DataFrame(data_json)
        
            # Configura el tamaño de la figura
            plt.figure(figsize=(10, 6))
        
            # Crea un gráfico de barras
            bars = plt.bar(df['hname'], df['reservation_count'], color='skyblue')
        
            # Añade título y etiquetas a los ejes
            plt.title('Top 10% de Hoteles con Más Reservaciones')
            plt.xlabel('Nombre del Hotel')
            plt.ylabel('Número de Reservaciones')
        
            # Rotar las etiquetas del eje X para mejor legibilidad
            plt.xticks(rotation=45, ha="right")
        
            # Añade los valores exactos encima de las barras
            for bar in bars:
                height = bar.get_height()
                plt.text(bar.get_x() + bar.get_width() / 2.0, height, '%d' % int(height), ha='center', va='bottom')
        
            # Ajusta el layout para asegurar que las etiquetas no se corten
            plt.tight_layout()
        
            # Muestra el gráfico
            plt.show()
        else:
            pass
            #print("Error en la respuesta:", response.status_code)

In [4]:

def graficar_capacidad_hoteles():
    with global_statistic_widget:
        url = 'http://127.0.0.1:5000/most/capacity'
        data = {
            "eid": 3
        }
        response = session.get(url, json=data, verify=False)
        # Verifica si la solicitud fue exitosa
        if response.status_code == 200:
            #print("Respuesta exitosa")
            data_json = response.json()
            # Convierte los datos JSON en un DataFrame de pandas
            df = pd.DataFrame(data_json)
            
            #global_statistic_widget.clear_output(wait=True)  # Limpiamos la salida anterior
            plt.figure(figsize=(10, 6))  # Tamaño del gráfico
            bars = plt.bar(df['hname'], df['total_capacity'], color='teal')  # Gráfico de barras
            
            # Añadimos etiquetas en las barras
            for bar in bars:
                yval = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2, yval + 1, int(yval), ha='center', va='bottom')
            
            plt.title('Top 5 Hoteles con Mayor Capacidad de Clientes')  # Título del gráfico
            plt.xlabel('Nombre del Hotel')  # Etiqueta eje X
            plt.ylabel('Capacidad Total')  # Etiqueta eje Y
            plt.xticks(rotation=45, ha='right')  # Rotamos las etiquetas para que sean legibles
            plt.tight_layout()  # Ajuste del layout
            plt.show()

In [5]:


# El widget Label para mostrar mensajes
mensaje_label = widgets.Label()
# Aquí está tu función actualizada para verificar el acceso y mostrar gráficos si es necesario
def actualizar_interfaz_global_stats(cookies):
    if not cookies:
        mensaje_label.value = "Necesitas hacer log in para ver la información en este área."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with global_statistic_widget:
            global_statistic_widget.clear_output()
    elif cookies.get('position', '') == 'Administrator':
        mensaje_label.value = "Gráficos Globales"
        # Mostrar los gráficos porque el usuario es un administrador
        mostrar_graficos_most_reservation()
        graficar_capacidad_hoteles()
    else:
        mensaje_label.value = "No tienes permiso para ver esta sección."
        # Asegurarse de que el área de gráficos esté vacía o escondida
        with global_statistic_widget:
            global_statistic_widget.clear_output()

In [6]:


submit_login = widgets.Output()
# Widget para mostrar la información almacenada en las cookies



username_input = widgets.Text(
    value='',
    placeholder='Ingrese su nombre de usuario',
    description='Usuario:',
    disabled=False
)

password_input = widgets.Password(
    placeholder='Ingrese su contraseña',
    description='Contraseña:',
    disabled=False
)

login_button = widgets.Button(
    description='Iniciar sesión',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Enviar',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

session = requests.Session()

@submit_login.capture(clear_output=True)
def on_login_button_clicked(b):
    # Aquí usarías `requests` para enviar los datos al servidor
    url = 'http://127.0.0.1:5000/login/in/frontend'
    data = {
        'username': username_input.value,
        'password': password_input.value
    }
    response = session.post(url, json=data, verify=False)  # `session` debería ser una instancia de `requests.Session()` ya creada
    
    # Ahora, maneja la respuesta
    if response.status_code == 200:
        print("Login exitoso")
        # Actualizar la información de las cookies
        employee_name_widget.value = "Bienvenido " + str(session.cookies.get_dict()['fname'] + " " + session.cookies.get_dict()['lname'] + " ") + "al Sistema Analítico Hotelero"
        # Aquí podrías actualizar el notebook o hacer algo con la respuesta
        actualizar_interfaz_global_stats(session.cookies.get_dict())
        username_input.close()
        password_input.close()
        login_button.close()
    else:
        print("Usuario o contaseña incorrecta")


login_button.on_click(on_login_button_clicked)
display(username_input, password_input, login_button, submit_login)



Text(value='', description='Usuario:', placeholder='Ingrese su nombre de usuario')

Password(description='Contraseña:', placeholder='Ingrese su contraseña')

Button(description='Iniciar sesión', icon='check', style=ButtonStyle(), tooltip='Enviar')

Output()

In [7]:
#Seccion para hacer log in
funcs = Functions()
funcs.create_styled_section(section_titles[0][0], section_titles[0][1])


HTML(value='\n        <style>\n        #login {\n          color: white; /* Cambia el color del texto */\n    …

In [8]:
#seccion para mostrar los local stats
funcs = Functions()
funcs.create_styled_section(section_titles[1][0], section_titles[1][1])


HTML(value='\n        <style>\n        #local_statistics {\n          color: white; /* Cambia el color del tex…

In [9]:
#seccion para mostrar los global stats
funcs = Functions()
funcs.create_styled_section(section_titles[2][0], section_titles[2][1])

            
display(mensaje_label)
display(global_statistic_widget)
            
actualizar_interfaz_global_stats(session.cookies.get_dict())




HTML(value='\n        <style>\n        #global_statistics {\n          color: white; /* Cambia el color del te…

Label(value='')

Output()

In [10]:
#seccion para crear nuevos recors
funcs = Functions()
funcs.create_styled_section(section_titles[3][0], section_titles[3][1])



HTML(value='\n        <style>\n        #create_records {\n          color: white; /* Cambia el color del texto…

In [11]:
#sexxion para borrar y actualizar nuevos records
funcs = Functions()
funcs.create_styled_section(section_titles[4][0], section_titles[4][1])


HTML(value='\n        <style>\n        #update_delete_records {\n          color: white; /* Cambia el color de…